In [7]:
def my_function(fname, lname):
    print(fname, 'Hello from a function', lname)

my_function('louie', 'roberts')

louie Hello from a function roberts


In [8]:
def kid_functions(*kids):
    print('the younges thild is '+ kids[-1])

kid_functions("emma", "knox", "louie")

the younges thild is louie


In [9]:
def kid_functions_2(child3, child2, child1):
    print('the younges thild is ' + child3)

kid_functions_2(child1='emma', child2='knox', child3='louie')

the younges thild is louie


In [10]:
def kid_functions_3(**kid):
    print('His last name is ' + kid['lname'])

kid_functions_3(fname = 'Tobias', lname = 'Refsnes')

His last name is Refsnes


In [15]:
def add_function(integer):
    for x in integer:
        sum = x + 10
    return sum

In [23]:
add_function([12])

22

In [18]:
add_function([1, 2])

12


In [20]:
integer_list = [1, 2]

add_function(integer_list)

12

In [24]:
results_list = []
for i in integer_list:
    sums = add_function([i])
    results_list.append(sums)

results_list

[11, 12]

### Tried to do a loop and function to process all the grassland climate data at once:

In [ ]:
results_list = []
rcp_list = [45, 85]
row_list = [0, 1, 2, 3]
for rcp in rcp_list:
    print(rcp)
    for row in row_list:
        print(row)
        grassland_rcp_annual_precip_da = process_climate_da(rcp, maca_df, row)
        print(grassland_rcp_annual_precip_da)
        results_list.append(grassland_rcp_annual_precip_da)

results_list